In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

In [4]:
import os
import time
import openpyxl
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
#from random import randrange  
import sys

**Scraping code 
  looping through pages of ads, scraping and formating data of interest (mainly numerical and some categorical)**

In [8]:
baseURL = "https://www.oglasnik.hr/prodaja-automobila?page="
koliko_manjkavih=[0]*12                  #koliko_manjkavih[i] = counts ads with i missing (relevant) fields ie koliko_manjkavih[0] counts 
                                         #ads with NO missing fields, koliko_manjkavih[1] counts those with one missing field etc
data = []

for i in range(1,4): #227):  #[randrange(100)]:                   #randrange was used while I debugged the scraping loop, not to skew the numberofviews too much
                                                            #also to try and catch as much different unexpected errors
    t0=time.time()          #measured time to optimise a bit
    print('\n',i,end=':')   #progress indicator            
    url = baseURL+str(i)                    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')    #render whole URL to BeautifulSoup type object
    soup = soup.find('div', class_='oglasnik-box')           #narrow down the view because all I need is in this 'div'
   
    #print(sys.getsizeof(soup),end="   ")   
    #  another exercise in optimisation, checking if I saved some space, seems not. 
    # too sidequest-y, later
    ads = soup.find_all('a', class_='classified-box')  
    #print(sys.getsizeof(ads),end="   ")

    t1=time.time()   
    print(f"page {t1-t0:.3f}s;",end="..")
    t0=t1    
    for j, ad in enumerate(ads, 1):         
         print(j,end=',')   #progress indicator
         url = ad.get('href')
         count_na_cells=0    #counter for missing (semi)important attributes. 
         #t0=time.time()   
         
         response = requests.get(url)
         soup = BeautifulSoup(response.content, 'html.parser')
         pom_tag=soup.find('h1', class_='h2-like')
         if(pom_tag):   title = pom_tag.text.strip()        #attribute 'title' 
         else : continue                                    #there can be a ghost ad - deleted and it still lists in outer loop
         """ t1=time.time()   
         print(f"title {t1-t0:.3f}s;",end="")
         t0=t1 """  #timer ...... turns out - as expected - that rendering soup is most time consuming 
         pom_tag=soup.select_one ('li[data-fullscreen-url]')
         if(pom_tag) : img_url=("https://www.oglasnik.hr/"+pom_tag['data-fullscreen-url'])  
         else : img_url=""                                                  
          #url to the first photo of the listed vehicle. I allow records with no image 
                   

         seller=""
         location=""
         # I need some categorical variables too. One course of action would be to compare small sellers from used car agencies
         
         pom_tag=soup.find('div',class_='top-details').contents[1]     # i guess '=' means its constructing an object for comparison?         
         if(pom_tag.name=='div'):                              
             seller=(pom_tag.text.strip().split('(')[0])       #keep only part of the string before the parenthesis      
             pom_tag=(pom_tag.next_sibling.next_sibling)       #.next_sibling method goes without parenthesis       
         if(pom_tag.name=='p') : location=(pom_tag.text.strip().split(',')[-1])
        
                 
         
         pom_tag=soup.find('dl',class_="price")
         if(pom_tag):
             price_EUR=(pom_tag.contents[3].text.strip().split()[0])  
             price_EUR=re.sub('\.', '', price_EUR)             # source has price in a form "178.944,38" - we need dot as decimal point and no comma
             price_EUR=re.sub('\,','.',price_EUR)              #remove dots as delimiter for thousands, and then replace decimal comma with a dot.
                                                               # this would have been faster with two chained .replace() methods
             price_kn=(pom_tag.contents[5].text.strip().split()[0])
             price_kn=re.sub('\.', '', price_kn)
             price_kn=re.sub('\,','.',price_kn)
         else: continue                     #some listings dont list asking price - that is too important attribute and such listings are dropped    
        
         
         pom_tag=soup.find('span',class_='ad-meta-single visible-xs visible-sm')
         nofviews=int(pom_tag.contents[4].text.strip().split()[0] )       
                             #I needed numerical attributes. May not be a relevant attribute. I'll do some statistical tests and see. 
         
         
         pom_tag=soup.find('span', string='Godina proizvodnje:')
         if(pom_tag): 
             pom_tag=pom_tag.find_parent('div')    
             year_of_prod=pom_tag.contents[2].strip()
         else:              
             continue     # another "dealbreaker" attribute - if there's no year of production listed, i skip that ad  
        
         pom_tag=pom_tag.find_parent()
         pom_tag2=pom_tag.find('span',string="Kilometraža:")
         if(pom_tag2):
             kms=  (pom_tag2.find_parent().contents[2].strip().split()[0]  )            
         else: 
             count_na_cells+=1   #semiimportant attribute, not dealbreaker
             kms=""

         pom_tag2=pom_tag.find('span',string="Snaga motora:")
         if(pom_tag2):
             kWs=  (pom_tag2.find_parent().contents[2].strip().split()[0]   )  
             #kWs= """ int """(pom_tag2.find_parent().contents[2].strip().split()[0]   )   
             #ValueError: invalid literal for int() with base 10: '51.5'
             #na 49. stranici oglasnika, na kvarat puta, ovaj error       
         else: 
             count_na_cells+=1
             kWs=""

                 
         
         pom_tag2=pom_tag.find('span',string="Proizvođač:")
         if(pom_tag2):manuf= pom_tag2.find_parent().contents[2].strip()
         else:
             manuf=""
         if (manuf==""): manuf=title.split()[0]   #dirty trick to fill some missing fields - first word in title is usually the manufacturer name

         pom_tag2=pom_tag.find('span',string="Model:")
         if(pom_tag2):model= pom_tag2.find_parent().contents[2].strip()
         else: 
             model=""
           #very few ads had this attribut correctly listed
         if (manuf == model): model=""
         
         pom_tag2=pom_tag.find('span',string="Radni obujam:")
         if(pom_tag2):cc= (pom_tag2.find_parent().contents[2].strip().split()[0])    #.split()[0] cuts of the measuring unit, only keeps first part of the string which is the number
         else: 
             count_na_cells+=1
             cc=""
         
         koliko_manjkavih[count_na_cells]+=1        #update how good is the 

         if (1): # ad je dobar: #pospremi_podatke  #(count_na_cells<2 ili 3)
              data.append({
                'title': title, 
                'year_of_prod': year_of_prod,
                'price_EUR': price_EUR,
                'price_kn': price_kn,
                'Mileage_kms':kms,
                'kWs':kWs,
                'cc': cc,
                'nofviews': nofviews,
                'manufacturer': manuf,   #problem. mozda se moze iz titlea izvuci.
                'model': model,          #problem
                'seller':seller,
                'location':location,
                'ad_URL': url,
                'image': img_url
                #'srcID': srcid,                
                #'PublishDate':lastActivated
            })
         t1=time.time()   
         print(f"..inner loop  {t1-t0:.3f}s;", end=' ' )
         t0=t1
         
df = pd.DataFrame(data)
#display(df)
    
    
#entire scape took 90ak minutes, for 21*226 pages. is that slow or normal? my guess normal
#seems slower when I run it from browser tab than from vsc


 1:page 1.156s;..1,..inner loop  0.789s; 2,..inner loop  0.593s; 3,..inner loop  1.773s; 4,..inner loop  0.826s; 5,..inner loop  1.003s; 6,..inner loop  0.643s; 7,..inner loop  1.073s; 8,..inner loop  1.579s; 9,..inner loop  1.835s; 10,..inner loop  0.895s; 11,..inner loop  1.791s; 12,..inner loop  1.037s; 13,..inner loop  0.670s; 14,..inner loop  0.855s; 15,..inner loop  1.301s; 16,..inner loop  0.631s; 17,..inner loop  0.992s; 18,..inner loop  0.436s; 19,..inner loop  0.497s; 20,..inner loop  0.967s; 
 2:page 0.499s;..1,..inner loop  0.797s; 2,..inner loop  0.735s; 3,..inner loop  1.221s; 4,..inner loop  1.044s; 5,..inner loop  1.141s; 6,..inner loop  1.058s; 7,..inner loop  1.517s; 8,..inner loop  1.473s; 9,..inner loop  1.262s; 10,..inner loop  0.967s; 11,..inner loop  1.295s; 12,..inner loop  0.472s; 13,..inner loop  0.655s; 14,..inner loop  1.056s; 15,..inner loop  1.443s; 16,..inner loop  0.416s; 17,..inner loop  1.168s; 18,..inner loop  0.477s; 19,..inner loop  0.983s; 20,..in

##### One thing to implement next time - would've been prudent to every so often (eg every 50-200 rows) append the partial scraped data to csv/xlsx/db  so if the script gets an error, big chunk of work is backed up 

In [ ]:

print(koliko_manjkavih)
print(len(data))
df = pd.DataFrame(data)
display(df)
df.to_csv("raw_data.csv", sep='|', encoding='utf-8',index=False)
df.to_excel('raw_data.xlsx' ,engine='xlsxwriter',index=False)   #backing up the dataframe to csv and excel



###### there was a time gap between these cells so have to read the dataframe back from csv to store it to database

In [1]:
import pandas


df = pandas.read_csv("raw_data_2.csv", encoding="utf-8", delimiter="|")
#ocitam raw podtke iz csv-a da ih spremim u bazu jer mi nisu ostali u runtimeu
len(df)     #df.head(14)


4491

###### geting info on type of attributes and on length of 'object' types/ strings / varchars so I know how much space to allocate in database

In [30]:
#df.info()    #columns 1-6 were all strings during scraping, but somewhere along the path scrape->csv->dataframe, got implicitly converted to numerical 
 
s = (df.dtypes == 'object')    
print(df.dtypes)
object_cols = list(s[s].index)
print(object_cols)
for i in object_cols : print(i, "    " , max(df[i].apply(lambda x:len(str(x)))))  #da vidim koliko da uzmem mjesta u bazi
 

title            object
year_of_prod    float64
price_EUR       float64
price_kn        float64
Mileage_kms     float64
kWs             float64
cc              float64
nofviews          int64
manufacturer     object
model            object
seller           object
location         object
ad_URL           object
image            object
dtype: object
['title', 'manufacturer', 'model', 'seller', 'location', 'ad_URL', 'image']
title      80
manufacturer      13
model      7
seller      44
location      43
ad_URL      136
image      121


######  Write raw data to database:

In [6]:
import psycopg2
from psycopg2 import sql
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="sibin",
    user="sibinuser",
    password="sib123"
)



df = pandas.read_csv("raw_data_2.csv", encoding="utf-8", delimiter="|")
#  
cur = conn.cursor()
#
cur.execute("DROP TABLE IF EXISTS oglasi.raw_data") 
cur.execute("""create TABLE oglasi.raw_data ( title varchar(90), year_of_prod integer, price_EUR numeric, price_kn numeric,      
                   Mileage_kms numeric, kWs numeric, cc numeric, nofviews integer,           
                             manufacturer varchar(30), model varchar(30),seller varchar(50), location varchar(50), ad_URL varchar(150), image varchar(150))""") 
#df.to_sql('oglasi.raw_data', con=conn, index_label='Index')   #to je ako se koristi sql alchemy

for row in df.reset_index().to_dict('rows'):
    #query ="""INSERT into oglasi.raw_data (title, year_of_prod, price_EUR,price_kn,Mileage_kms,kWs, cc , nofviews , manufacturer , model ,seller , location , ad_URL , image) 
    #             values('%s',%i,%f,%f,%f,%f,%f,%i,'%s','%s','%s','%s','%s','%s');""" %  (row['title'], row['year_of_prod'], row['price_EUR'], row['price_kn'], row['Mileage_kms'], row['kWs'], row['cc'], row['nofviews'], row['manufacturer'], row['model'], row['seller'], row['location'], row['ad_URL'], row['image'])
    
    #        above was my naive idea of forming query as a string - trouble came from empty fields in numerical attributes. a colleague suggested the solution below
    query=sql.SQL("insert into oglasi.raw_data VALUES ({},{},{},{},{},{},{}, {}, {}, {}, {}, {}, {}, {})").format(
                        sql.Literal(str(row['title'])), sql.Literal(row['year_of_prod']), sql.Literal(row['price_EUR']), sql.Literal(row['price_kn']), 
                        sql.Literal(row['Mileage_kms']),sql.Literal(row['kWs']), sql.Literal(row['cc']), sql.Literal(row['nofviews']), 
                        sql.Literal(str(row['manufacturer'])), sql.Literal(str(row['model'])), sql.Literal(str(row['seller'])), sql.Literal(str(row['location'])),
                        sql.Literal(str(row['ad_URL'])), sql.Literal(str(row['image'])))
    #print(query)
    cur.execute(query) 
 
# Execute a sample SQL query
 
cur.execute("SELECT * FROM oglasi.raw_data")

# Fetch the query result
result = cur.fetchall()

# Print the query result
print(result)
 
conn.commit()
# Close the cursor and connection objects
cur.close()
conn.close()

C:\Users\Admin\AppData\Local\Temp\ipykernel_7044\4024512335.py:24: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  for row in df.reset_index().to_dict('rows'):


[('MAZDA CX-30 e-X186 GT PLUS', 2022, Decimal('30959.35'), Decimal('233263.22'), Decimal('NaN'), Decimal('137.0'), Decimal('1998.0'), 18, 'MAZDA', 'nan', 'Auto Benussi ', ' Istarska', 'https://www.oglasnik.hr/prodaja-automobila/mazda-cx-30-e-x186-gt-plus-oglas-5264750', 'https://www.oglasnik.hr//repository/images/_var/f/3/f30c8f5e1394492ef06ffd26aa5ce75801234567-GalleryFullscreen.jpg'), ('RENAULT MEGANE GRANDTOUR LIMITED ENERGY', 2018, Decimal('13900.0'), Decimal('104729.55'), Decimal('136013.0'), Decimal('80.0'), Decimal('1461.0'), 13, 'RENAULT', 'nan', 'Auto Benussi ', ' Grad Zagreb', 'https://www.oglasnik.hr/prodaja-automobila/renault-megane-grandtour-limited-energy-oglas-5264747', 'https://www.oglasnik.hr//repository/images/_var/b/f/bfa47b1077b18aaafe56a43e2360653001234567-GalleryFullscreen.jpg'), ('RENAULT MEGANE GRANDTOUR LIMITED ENERGY', 2018, Decimal('13900.0'), Decimal('104729.55'), Decimal('132128.0'), Decimal('80.0'), Decimal('1461.0'), 13, 'RENAULT', 'nan', 'Auto Benussi ',

###### write cleaned data to database.  (cleaning done in the other, "analytics", notebook) 

In [8]:
import psycopg2
from psycopg2 import sql
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="sibin",
    user="sibinuser",
    password="sib123"
)



df = pandas.read_csv("data_cleaned_from_outliers.csv", encoding="utf-8", delimiter="|")
# Create a cursor object to execute SQL queries
cur = conn.cursor()
#
cur.execute("DROP TABLE IF EXISTS oglasi.data_cleaned") 
cur.execute("""create TABLE oglasi.data_cleaned ( title varchar(90), age numeric, price_EUR numeric,  
                   Mileage_kms numeric, kWs numeric, cc numeric, nofviews integer,           
                             manufacturer varchar(30),  seller varchar(50), location varchar(50), ad_URL varchar(150), image varchar(150))""") 
#df.to_sql('oglasi.raw_data', con=conn, index_label='Index')   #to je ako se koristi sql alchemy

for row in df.reset_index().to_dict('rows'):
    #query ="""INSERT into oglasi.raw_data (title, age, price_EUR,price_kn,Mileage_kms,kWs, cc , nofviews , manufacturer , model ,seller , location , ad_URL , image) 
    #             values('%s',%i,%f,%f,%f,%f,%f,%i,'%s','%s','%s','%s','%s','%s');""" %  (row['title'], row['year_of_prod'], row['price_EUR'], row['price_kn'], row['Mileage_kms'], row['kWs'], row['cc'], row['nofviews'], row['manufacturer'], row['model'], row['seller'], row['location'], row['ad_URL'], row['image'])
    query=sql.SQL("insert into oglasi.data_cleaned VALUES ({},{},{},{},{}, {}, {}, {}, {}, {}, {}, {})").format(
                        sql.Literal(str(row['title'])), sql.Literal(row['age']), sql.Literal(row['price_EUR']),  
                        sql.Literal(row['Mileage_kms']),sql.Literal(row['kWs']), sql.Literal(row['cc']), sql.Literal(row['nofviews']), 
                        sql.Literal(str(row['manufacturer'])),   sql.Literal(str(row['seller'])), sql.Literal(str(row['location'])),
                        sql.Literal(str(row['ad_URL'])), sql.Literal(str(row['image'])))
    #print(query)
    cur.execute(query) 
 
 
#cur.execute("SELECT * FROM oglasi.raw_data")

# Fetch the query result
result = cur.fetchall()

# Print the query result
print(result)
 
conn.commit()
# Close the cursor and connection objects
cur.close()
conn.close()

C:\Users\Admin\AppData\Local\Temp\ipykernel_7044\2314089960.py:24: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  for row in df.reset_index().to_dict('rows'):


[('MAZDA CX-30 e-X186 GT PLUS', 2022, Decimal('30959.35'), Decimal('233263.22'), Decimal('NaN'), Decimal('137.0'), Decimal('1998.0'), 18, 'MAZDA', 'nan', 'Auto Benussi ', ' Istarska', 'https://www.oglasnik.hr/prodaja-automobila/mazda-cx-30-e-x186-gt-plus-oglas-5264750', 'https://www.oglasnik.hr//repository/images/_var/f/3/f30c8f5e1394492ef06ffd26aa5ce75801234567-GalleryFullscreen.jpg'), ('RENAULT MEGANE GRANDTOUR LIMITED ENERGY', 2018, Decimal('13900.0'), Decimal('104729.55'), Decimal('136013.0'), Decimal('80.0'), Decimal('1461.0'), 13, 'RENAULT', 'nan', 'Auto Benussi ', ' Grad Zagreb', 'https://www.oglasnik.hr/prodaja-automobila/renault-megane-grandtour-limited-energy-oglas-5264747', 'https://www.oglasnik.hr//repository/images/_var/b/f/bfa47b1077b18aaafe56a43e2360653001234567-GalleryFullscreen.jpg'), ('RENAULT MEGANE GRANDTOUR LIMITED ENERGY', 2018, Decimal('13900.0'), Decimal('104729.55'), Decimal('132128.0'), Decimal('80.0'), Decimal('1461.0'), 13, 'RENAULT', 'nan', 'Auto Benussi ',

In [40]:
#vjezbica iz regexa kako brisati nezeljene znakove iz stringa,drugo je primjer sa stackoverflowa, prvo moja adaptacija na moj problem
d='178.944,38'
#display(df['Price_EUR'])
print(re.sub('\,', '.',re.sub('\.', '', d)))

line = 'Q: Do I write ;/.??? No!!!'
print(re.sub('\ |\?|\.|\!|\/|\;|\:', '', line))
print(line)

178944.38
QDoIwriteNo
Q: Do I write ;/.??? No!!!


In [84]:
#left to do:

#store df to snowflake

#scrape elsewhere for more samples of the same stuff (used cars) - index.hr, german site, etc.. 

#scrape something that has a bot restriction (njuskalo). below is an attempt with random pauses between requests, turns out not enough, still triggers the 429 code
       # Hrvoje mentioned playwright

import time
baseURL="https://geizhals.de/?cat=umtsover&pg=" #"https://www.burzanautike.com/hr/mali_oglasi/do_6_m/2,25/0_0_0_"                               
for i in [3,56,78,5,4,90]:
     url=baseURL+str(i)    
     print(url)
     response=requests.get(url)  #dobra praksa pocekati stavit pauzu jer instant scrape alarmira web da si bot
     print(response)
     time.sleep(15+randrange(5))  #15ak+ sec razmaka ga tripne nakon 3. requesta. 

https://geizhals.de/?cat=umtsover&pg=3
<Response [200]>
https://geizhals.de/?cat=umtsover&pg=56
<Response [200]>
https://geizhals.de/?cat=umtsover&pg=78
<Response [429]>


KeyboardInterrupt: 